In [1]:
import numpy as np
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0])-np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle>180.0:
        angle=360-angle
    return angle

In [3]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            center_hips = [(left_hip[0]+right_hip[0])/2, (left_hip[1]+right_hip[1])/2]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            center_shoulders = [(left_shoulder[0]+right_shoulder[0])/2, (left_shoulder[1]+right_shoulder[1])/2]
            nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x, landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            right_ear = [landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y]
            left_ear = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x, landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
            left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
            
            #Calculate angles
            # l_neck r_neck neck_front l_shoulder r_shoulder l_elb r_elb l_hip r_hip c_hip l_knee r_knee l_ankle r_ankle
            angles = [calculate_angle(left_shoulder, center_shoulders, left_ear),
                         calculate_angle(right_shoulder, center_shoulders, right_ear),
                         calculate_angle(center_hips, center_shoulders, nose),
                         calculate_angle(left_elbow, left_shoulder, left_hip),
                         calculate_angle(right_elbow, right_shoulder, right_hip),
                         calculate_angle(left_wrist, left_elbow, left_shoulder),
                         calculate_angle(right_wrist, right_elbow, right_shoulder),
                         calculate_angle(left_shoulder, left_hip, left_knee),
                         calculate_angle(right_shoulder, right_hip, right_knee),
                         calculate_angle(left_knee, center_hips, right_knee),
                         calculate_angle(left_hip, left_knee, left_ankle),
                         calculate_angle(right_hip, right_knee, right_ankle),
                         calculate_angle(left_knee, left_ankle, left_foot_index),
                         calculate_angle(right_knee, right_ankle, right_foot_index)
            ]
            
            print(angles)
            
        except:
            pass
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        cv2.imshow("Mediapipe Feed", image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

[76.38949252705659, 71.66344379097364, 174.91826571570982, 23.34987790142689, 17.207407271438843, 157.1578115376988, 157.94827034123827, 174.01062573676197, 175.81792898896077, 25.980674816895032, 176.90027992487256, 178.13838413714785, 166.3304970275487, 168.660367129253]
[75.82777570017336, 72.57622206227731, 175.13888239265754, 22.972186416092534, 17.05520800519203, 159.1517411259703, 161.90476319218078, 173.11782382686374, 177.65449567494892, 23.72889235499856, 177.31207195627513, 179.4459936358061, 165.48108983676022, 169.39444445027323]
[75.71621597848879, 72.9957936435352, 175.86545744555997, 23.25347354745796, 16.903749651034666, 159.29926902037295, 161.63490570985914, 173.2206336830236, 177.36642704625297, 24.203337521145308, 177.33143534739244, 179.2052612263874, 165.52168821044987, 168.84237928285614]
[75.78828330140615, 73.15862679965424, 176.21987909974126, 23.37683275253835, 16.888950945432132, 159.5981170715932, 160.56626302686146, 173.22146924376187, 177.06842368682416,

[78.10113091236111, 74.51523047387208, 179.25186631545375, 21.52293946648309, 19.296629615757535, 165.84396302226892, 161.18109656608232, 174.76128544305627, 176.98778330198806, 22.720382761139696, 179.05593868157055, 179.9202918378618, 165.14190801981624, 169.1541618841137]
[76.66379240467666, 75.42750002344874, 178.02359568572746, 21.49667414093482, 19.05855296674337, 166.34211368972234, 161.48768925851007, 174.7712583086425, 176.87004934764187, 23.06744994479792, 179.2027659594405, 179.3196880298588, 163.92756825576322, 167.86421287114737]
[75.91285913238383, 76.04955427799422, 176.1109275242532, 21.317211511415596, 18.83951190306508, 165.59151963568175, 162.6018292069854, 174.55945643476335, 176.88686957400867, 23.581347569074456, 178.60894721757495, 179.84979059434883, 163.59464083389116, 167.29481553800005]
[76.29471615097593, 75.12983698401818, 176.630610333338, 21.94014563005592, 19.320863964073204, 165.47181332591234, 154.93302348252422, 174.44122755563265, 176.42896868260678,

[75.67595602300892, 75.85061848593557, 175.20147077248234, 24.445002974225655, 20.60258007634578, 159.37148252757515, 145.3523013147509, 173.19830865771388, 177.97339586074574, 24.57005403824594, 177.88712888902307, 179.92453712919234, 165.224971300431, 166.84227995240744]
[75.6423293884014, 75.88743567434376, 175.19959832522386, 24.455963414320244, 20.794251489668945, 159.12129847499418, 149.30470640586813, 173.27498096346199, 177.77229992176848, 24.700545845936958, 177.9178510843462, 179.9869160234715, 165.11435450663924, 165.79358476957566]


In [4]:
angles

[75.6423293884014,
 75.88743567434376,
 175.19959832522386,
 24.455963414320244,
 20.794251489668945,
 159.12129847499418,
 149.30470640586813,
 173.27498096346199,
 177.77229992176848,
 24.700545845936958,
 177.9178510843462,
 179.9869160234715,
 165.11435450663924,
 165.79358476957566]